In [1]:
"""
Connect senior:Mainland
"""
import pandas as pd
import json
import requests
def ssql(sql):
    admin_token = '2cebeb5c7edac5ef0d7c5a640e69fc7d43bf2cd6a24ce43dfee3dd33662c0bab'
    url = 'http://sensor.wb-intra.com/api/sql/query?token=%s&project=production' % admin_token
    data = {'q': sql, 'format': 'json'}
    req = requests.post(url,data)
    req_dec = req.content.decode()
    try:
        req_json = json.loads('[' + req_dec.replace('\n', ',')[:-1] + ']')
        df_d_id = pd.DataFrame(req_json)
        return df_d_id
    except:
        print(req_dec)

In [3]:
df = ssql("""
SELECT appId,
       sum(pay) AS pay
FROM events
WHERE event = 'pay'
  AND appId IN ('20009',
                '20014',
                '30015')
  AND date = current_date() - interval 1 DAY
GROUP BY 1
""")

In [4]:
df.reset_index()

,appid,pay
0,30015,468.00
1,20014,1868.09


In [16]:
df[df['appid'] == '20014'].reset_index(drop = True).loc[0,'pay']

1868.09

In [17]:
df[df['appid'] == '30015'].reset_index(drop = True).loc[0,'pay']

468.0

In [6]:
df['pay'].sum()

2336.09

In [7]:
df_count = ssql("""
SELECT appId,
       count(distinct distinct_id) as `付费人数`
FROM events
WHERE event = 'pay'
  AND appId IN ('20009',
                '20014',
                '30015')
  AND date = current_date() - interval 1 DAY
GROUP BY 1
""")

In [26]:
df_count

,付费人数,appid
0,19,30015
1,66,20014


In [20]:
len(df_count[df_count['appid'] == '20009'])

0

In [30]:
df_count[df_count['appid'] == '20014']

,付费人数,appid
1,66,20014


In [38]:
c =[df_count[df_count['appid'] == '20009'].reset_index(drop = True).loc[0,'付费人数'] if len(df_count[df_count['appid'] == '20009']) != 0 else '暂无数据'][0]

In [39]:
c

'暂无数据'